In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(dotenv_path="../.env")

open_api_key = os.getenv("open_api_key")

openAI_params = {
    'api_key': open_api_key
}
client = OpenAI(**openAI_params)

#### Streaming API response

By default, when you make a request to the OpenAI API, we generate the model's entire output before sending it back in a single HTTP response. 
When generating long outputs, waiting for a response can take time. Streaming responses lets you start printing or processing the beginning of the model's output while it continues generating the full response.

In [22]:
stream = client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "user",
            "content": "Say 'double bubble bath' ten times fast.",
        },
    ],
    stream=True, # Enable this for streaming response
)

types_of_events = {}
response = ""
for event in stream:
    type_ = event.type # To get the type of response
    if not type_ in types_of_events:
        types_of_events[type_] = 1
    else:
        types_of_events[type_] += 1

    if event.type == "response.output_text.delta":
        response += event.delta
        print(event.delta, end="")
print("\n", response)
{k: v for k, v in sorted(types_of_events.items(), key=lambda x: x[1], reverse=True)}

That’s a fun tongue twister! It’s quite a challenge to say "double bubble bath" ten times quickly without stumbling. Give it a try and see how you do!
 That’s a fun tongue twister! It’s quite a challenge to say "double bubble bath" ten times quickly without stumbling. Give it a try and see how you do!


{'response.output_text.delta': 36,
 'response.created': 1,
 'response.in_progress': 1,
 'response.output_item.added': 1,
 'response.content_part.added': 1,
 'response.output_text.done': 1,
 'response.content_part.done': 1,
 'response.output_item.done': 1,
 'response.completed': 1}

#### Description of Common Responses
1. response.created: An event that is emitted when a response is created.
2. response.in_progress: Emitted when the response is in progress.
3. response.completed: Emitted when the model response is complete.
4. response.failed: An event that is emitted when a response fails.
5. response.incomplete: An event that is emitted when a response finishes as incomplete.
6. response.output_item.added: Emitted when a new output item is added.
7. response.output_item.done: Emitted when an output item is marked done.
8. response.content_part.added: Emitted when a new content part is added.
9. response.content_part.done: Emitted when a content part is done.
10. response.output_text.delta: Emitted when there is an additional text delta.
11. response.output_text.annotation.added: Emitted when a text annotation is added.
12. response.output_text.done: Emitted when text content is finalized.
13. response.refusal.delta: Emitted when there is a partial refusal text.
14. response.refusal.done: Emitted when refusal text is finalized.
15. error: Emitted when an error occurs.

#### Function Calling with Streaming Output

In [21]:
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            }
        },
        "required": [
            "location"
        ],
        "additionalProperties": False
    }
}]

stream = client.responses.create(
    model="gpt-4o",
    input=[{"role": "user", "content": "What's the weather like in Paris today?"}],
    tools=tools,
    stream=True
)

types_of_events = {}
response = ""
for event in stream:
    type_ = event.type # To get the type of response
    if not type_ in types_of_events:
        types_of_events[type_] = 1
    else:
        types_of_events[type_] += 1

    if event.type == "response.function_call_arguments.delta":
        response += event.delta
        print(event.delta, end="")

print("\n", response)
{k: v for k, v in sorted(types_of_events.items(), key=lambda x: x[1], reverse=True)}

{"location":"Paris, France"}
 {"location":"Paris, France"}


{'response.function_call_arguments.delta': 7,
 'response.created': 1,
 'response.in_progress': 1,
 'response.output_item.added': 1,
 'response.function_call_arguments.done': 1,
 'response.output_item.done': 1,
 'response.completed': 1}

#### Structured Output with Streaming response

In [31]:
stream = client.responses.create(
    model="gpt-4o",
    input=[
        {"role": "system", "content": "Extract entities from the input text"},
        {
            "role": "user",
            "content": "The quick brown fox jumps over the lazy dog with piercing blue eyes"
        },
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "entities",
            "schema": {
                "type": "object",
                "properties": {
                    "attributes": {
                        "type": "array",
                        "items": {"type": "string"}
                    },
                    "colors": {
                        "type": "array",
                        "items": {"type": "string"}
                    },
                    "animals": {
                        "type": "array",
                        "items": {"type": "string"}
                    }
                },
                "required": ["attributes", "colors", "animals"],
                "additionalProperties": False
            },
        }
    },
    stream=True,
)

types_of_events = {}
response = ""
for event in stream:
    type_ = event.type # To get the type of response
    if not type_ in types_of_events:
        types_of_events[type_] = 1
    else:
        types_of_events[type_] += 1

    if event.type == "response.output_text.delta":
        response += event.delta
        print(event.delta, end="")
    elif event.type == 'response.completed':
        print("\nCompleted")
        print(event.response.output[0].to_json())

print("\n", response)
{k: v for k, v in sorted(types_of_events.items(), key=lambda x: x[1], reverse=True)}

{"attributes":["quick","brown","lazy","piercing","blue"],"colors":["brown","blue"],"animals":["fox","dog"]}
Completed
{
  "id": "msg_67d81a43da6c8191bacd0e8107c8b14a00abf201e220537f",
  "content": [
    {
      "annotations": [],
      "text": "{\"attributes\":[\"quick\",\"brown\",\"lazy\",\"piercing\",\"blue\"],\"colors\":[\"brown\",\"blue\"],\"animals\":[\"fox\",\"dog\"]}",
      "type": "output_text"
    }
  ],
  "role": "assistant",
  "status": "completed",
  "type": "message"
}

 {"attributes":["quick","brown","lazy","piercing","blue"],"colors":["brown","blue"],"animals":["fox","dog"]}


{'response.output_text.delta': 29,
 'response.created': 1,
 'response.in_progress': 1,
 'response.output_item.added': 1,
 'response.content_part.added': 1,
 'response.output_text.done': 1,
 'response.content_part.done': 1,
 'response.output_item.done': 1,
 'response.completed': 1}